In [ ]:
#import packages
import numpy as np
import pandas as pd
import datetime as dt
import os 

In [ ]:
#import page
import requests as rq
base_url = 'https://minimisj2.bge-hautsdefrance.fr/contactsLDAP_photo_modal.php'
#
id= '01717140066145957'
page = rq.get(f"{base_url}?id={id}", auth=('asibille', 'Electro22@AppliRH'))
list = page.content
list

In [ ]:
#Beautifulsoup
from bs4 import BeautifulSoup
list_bs = BeautifulSoup(list, 'html.parser')
list_bs

In [ ]:
#extract names list
list_name = list_bs.find_all('h1')
list_name = [name.get_text() for name in list_name]
list_name
len(list_name)

In [ ]:
#extract positions list
list_position = list_bs.find_all('h2')
list_position = [position.get_text() for position in list_position]
list_position
len(list_position)

In [ ]:
#extract agencies list
list_agency = list_bs.find_all('h3')
#avoid doubled names: Use list comprehension along with enumeration to filter out duplicate agency names
list_agency = [agency.get_text() for key, agency in enumerate(list_agency) if key % 2 == 0]
list_agency = [agency.replace("'", '"') for agency in list_agency]
list_agency

In [ ]:
#attempt to zip list
try:
    list(zip(list_name, list_position, list_agency))

except Exception as ex:
    print(f"An unexpected exception occurred: {ex}")
    # Handle other exceptions

In [ ]:
#create a Dataframe based on a previous created dictionary
global_list = {'name' : list_name,
               'position' : list_position,
               'agency' : list_agency}
global_list = pd.DataFrame(global_list)
global_list

In [ ]:

# Filter rows where two agencies are mentioned
mask = global_list['agency'].str.contains('\+')
global_list_rowsPlus = global_list.loc[mask]

global_list_unmasked = global_list[~mask]

len(global_list_rowsPlus)

new_rows = [] #create empty list for append new rows copying
rows_drop = [] #create empty list for append old rows to erase

for indexr, row in global_list_rowsPlus.iterrows():
    agencySplits = row['agency'].split('+')
    if len(agencySplits) > 1:
    #if len(grlps)
        for agency in agencySplits:
            new_row = row.copy()  # Make a copy of the original row
            new_row['agency'] = agency.strip()  # Update the agency
            new_rows.append(new_row) # Append the new row to the list
        rows_drop.append(indexr) # Append the index of rows to drop
            # global_list_rowsPlus.drop(index=indexr)  
    else:
        new_rows.append(row)
        # Duplicate rows by concatenating DataFrame with itself

# Create a new DataFrame with the modified rows based on global_list_rowsPlus
global_list_rowsPlus_modified = pd.DataFrame(new_rows, columns=global_list_rowsPlus.columns)

# Drop the rows to be dropped from the original DataFrame
global_list_rowsPlus.drop(index=rows_drop, inplace=True)

# concatenate lists updated
global_list_rowsPlus = pd.concat([global_list_rowsPlus, global_list_rowsPlus_modified])

#by concatening we add initial rows of global list with '+' 
global_list_duplicated = pd.concat([global_list_unmasked, global_list_rowsPlus], ignore_index=True)

#sort list
global_list_duplicated_sorted = global_list_duplicated.sort_values(['name', 'agency'])

global_list_duplicated_sorted


In [ ]:
#cleaning

!python -m pip install unidecode -U
import unidecode



# Define lambdas for transformations
lower_index = lambda x: x.lower() if isinstance(x, str) else x
erase_indent = lambda x: x.replace('-', ' ') if isinstance(x, str) else x
strip_index = lambda x: x.strip() if isinstance(x, str) else x
accents_strip = lambda x: unidecode.unidecode(x) if isinstance(x, str) else x   # Apply unidecode only to string values
erase_apostrophe = lambda x: x.replace("'", " ") if isinstance(x, str) else x

# Identify columns with dtype 'object' (string columns)
str_columns_list = [str_column for str_column in global_list_duplicated_sorted.columns if global_list_duplicated_sorted[str_column].dtype == 'object']

# Apply transformations to selected string columns
for column in str_columns_list:
    global_list_duplicated_sorted[column] = global_list_duplicated_sorted[column].apply(lower_index).apply(erase_indent).apply(strip_index).apply(accents_strip).apply(erase_apostrophe)

# Output the transformed DataFrame
global_list_duplicated_sorted

In [ ]:
global_list_duplicated_sorted.to_csv('people_global_list_05_2024.csv', index=False)

SANDBOX

In [ ]:
# Filter rows where two agencies are mentioned
mask = global_list['agency'].str.contains('\+')
global_list_rowsPlus = global_list.loc[mask]

# Duplicate rows by concatenating DataFrame with itself
global_list_duplicated = pd.concat([global_list, global_list_rowsPlus], ignore_index=True)

# Sort the duplicated DataFrame by 'agency'
global_list_duplicated_sorted = global_list_duplicated.sort_values(by=['name', 'agency'])

# Erase leading and trailing spaces in column 'agency'
global_list_duplicated_sorted['agency'] = global_list_duplicated_sorted['agency'].apply(lambda x: x.strip())

# Function to split and keep the left part for the first duplicated line
def splitLeft_replace(x):
    split_parts = x.split('+')
    il = len(split_parts)
    if il > 1:
        return split_parts[0]  # Keep the first part
    else:
        return x  # Return the original value if no '+' sign found

# Function to split and keep the right part for the second duplicated line
def splitRight_replace(x):
    split_parts = x.rsplit('+')
    il = len(split_parts)
    if il == 1:
        return split_parts[1 - il]  # Keep the last part
    elif il > 1:
        return split_parts[il - 1]
    else:
        return x  # Return the original value if no '+' sign found

def defineSplitSide(x):
# Iterate over the DataFrame to handle duplicated rows
    for i in range(len(x) - 1):  # Exclude the last row to prevent index out of range
        current_agency = x.iloc[i, -1]
        next_agency = x.iloc[i + 1, -1]
        if current_agency == next_agency:
            #if i % 2 == 0:  # Even index (first duplicated line)
                # Apply splitLeft_replace function to clean the 'agency' column
            x.iloc[i, -1] = splitLeft_replace(current_agency)
            
            #else:  # Odd index (second duplicated line)
                # Apply splitRight_replace function to clean the 'agency' column
            x.iloc[i + 1, -1] = splitRight_replace(next_agency)
        else:
            continue
    return global_list_duplicated_sorted    


defineSplitSide(global_list_duplicated_sorted)

masktest = global_list_duplicated_sorted['name'] == 'Arnaud SIBILLE' 
global_list_duplicated_sorted.loc[masktest]



In [ ]:
# Filter rows where two agencies are mentioned
mask = global_list['agency'].str.contains('\+')
global_list_rowsPlus = global_list[mask]

# Duplicate rows by concatenating DataFrame with itself
global_list_duplicated = pd.concat([global_list, global_list_rowsPlus], ignore_index=True)

# Sort the duplicated DataFrame by 'agency'
global_list_duplicated_sorted = global_list_duplicated.sort_values(by=['name', 'agency'])

# Function to split and keep the left part for the first duplicated line
def splitLeft_replace(x):
    split_parts = x.replace('+', ' ').split(' ')
    if len(split_parts) > 1:
        return split_parts[0]  # Keep the first part
    else :
        return x  # Return the original value if no '+' sign found

# Function to split and keep the right part for the second duplicated line
def splitRight_replace(x):
    split_parts = x.replace('+', ' ').split(' ')
    il = len(split_parts)
    if il > 1:
        return split_parts[il - 1]  # Keep the last part
    else:
        return x  # Return the original value if no '+' sign found


# Apply splitLeft_replace and splitRight_replace functions to clean the 'agency' column
## mistake : by first applying to all the column def leftreplace => the agency column contains only one word in every rows, so impossible after to apply def rightReplace 
global_list_duplicated_sorted['agency'] = global_list_duplicated_sorted['agency'].apply(splitLeft_replace)

# Filter the second duplicated row and apply splitRight_replace to clean the 'agency' column
second_duplicated_mask = global_list_duplicated_sorted[['name', 'agency']].duplicated()
global_list_duplicated_sorted.loc[second_duplicated_mask, 'agency'] = global_list_duplicated_sorted.loc[second_duplicated_mask, 'agency'].apply(splitRight_replace)

global_list_duplicated_sorted

masktest = global_list_duplicated_sorted['name'] == 'Arnaud SIBILLE' 
global_list_duplicated_sorted.loc[masktest]




In [ ]:
# Filter rows where two agencies are mentioned
mask = global_list['agency'].str.contains('\+')
global_list_rowsPlus = global_list.loc[mask]

# Duplicate rows by concatenating DataFrame with itself
global_list_duplicated = pd.concat([global_list, global_list_rowsPlus], ignore_index=True)

# Sort the duplicated DataFrame by 'agency'
global_list_duplicated_sorted = global_list_duplicated.sort_values(by=['name', 'agency'])

# Erase leading and trailing spaces in column 'agency'
global_list_duplicated_sorted['agency'] = global_list_duplicated_sorted['agency'].apply(lambda x: x.strip())

# Function to split and keep the left part for the first duplicated line
def splitLeft_replace(x):
    split_parts = x.split('+')
    il = len(split_parts)
    if il > 1:
        return split_parts[0]  # Keep the first part
    else:
        return x  # Return the original value if no '+' sign found

# Function to split and keep the right part for the second duplicated line
def splitRight_replace(x):
    split_parts = x.rsplit('+')
    il = len(split_parts)
    if il == 1:
        return split_parts[1 - il]  # Keep the last part
    elif il > 1:
        return split_parts[il - 1]
    else:
        return x  # Return the original value if no '+' sign found

def defineSplitSide(x):
# Iterate over the DataFrame to handle duplicated rows
    for i in range(len(x) - 1):  # Exclude the last row to prevent index out of range
        current_agency = x.iloc[i, -1]
        next_agency = x.iloc[i + 1, -1]
        if current_agency == next_agency:
            #if i % 2 == 0:  # Even index (first duplicated line)
                # Apply splitLeft_replace function to clean the 'agency' column
            x.iloc[i, -1] = splitLeft_replace(current_agency)
            
            #else:  # Odd index (second duplicated line)
                # Apply splitRight_replace function to clean the 'agency' column
            x.iloc[i + 1, -1] = splitRight_replace(next_agency)
        else:
            continue
    return global_list_duplicated_sorted    


defineSplitSide(global_list_duplicated_sorted)

masktest = global_list_duplicated_sorted['name'] == 'Arnaud SIBILLE' 
global_list_duplicated_sorted.loc[masktest]



In [ ]:
global_list_duplicated_sorted.to_csv('people_global_list.csv', index=False)

SELENIUM

In [ ]:
import selenium 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

#import packages dynamic scraping
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [ ]:
#dynamic website
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException


In [ ]:
try:
    service = Service()
    options = webdriver.ChromeOptions
    driver = webdriver.Chrome(service=service, options=option)
except Exception as ex:
    print(f"An unexpected exception occurred: {ex}")
    # Handle other exceptions

finally:
    if 'driver' in locals() and driver is not None:
        driver.quit()